In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/home/smoeding2/caches/'
os.environ['XDG_CACHE_HOME'] = '/home/smoeding2/caches/'

In [2]:
import math
from tqdm import tqdm
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import torch
import pandas as pd
import math
from sklearn.metrics import accuracy_score

/home/smoeding2/.conda/envs/dl/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
model_name = "saved_models/chunk_instruct_tokens"

In [4]:
tokenizer=AutoTokenizer.from_pretrained(model_name, padding_side='left')
tokenizer.pad_token_id=tokenizer.eos_token_id
tokenizer.padding_size="left"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/smoeding2/.conda/envs/dl/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Embedding(32008, 4096)

In [6]:
generation_config=GenerationConfig(max_new_tokens=100,
                                    temperature=0.2,
                                    top_p=0.95,
                                    top_k=40,
                                    repetition_penalty=1.2,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    do_sample=True,
                                    use_cache=True,
                                    output_attentions=False,
                                    output_hidden_states=False,
                                    output_scores=False,
                                    remove_invalid_values=True
                                    )

In [14]:
def prompt_model(prompt):
    input_tokens=tokenizer(f"Du bist ein hilfreicher Assistent, der alle Fragen so korrekt wie möglich beantwortet. <BENUTZER>: {prompt} <ASSISTENT>: BEGINANTWORT", return_tensors="pt").to(model.device)
    output_tokens=model.generate(**input_tokens, generation_config=generation_config, pad_token_id=tokenizer.eos_token_id)[0]
    answer=tokenizer.decode(output_tokens)
    return answer

In [15]:
prompt_case = "Es folgt eine Fallbeschreibung, dessen Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Die Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Beantworte die Frage, indem du die korrekte Antwortmöglichkeit wiedergibst."
prompt_no_case = "Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Die Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Beantworte die Frage, indem du die korrekte Antwortmöglichkeit wiedergibst."

In [16]:
questions_df = pd.read_csv("output.csv").head(5)
questions_df["Prediction LLM"] = [-1 for i in range(len(questions_df))]
questions_df["Answer LLM"] = [-1 for i in range(len(questions_df))]

In [17]:
from transformers.utils import logging

logging.set_verbosity_info()

In [18]:
for index, row in tqdm(questions_df.iterrows()):
    if row["Answer LLM"] == -1:
        if type(row["Case"]) == str:
            prompt = prompt_case.replace("<INSERTFALL>",row["Case"])
        else:
            prompt = prompt_no_case
        prompt = prompt.replace("<INSERTFRAGE>",row["Question"])
        for i in range(5):
            prompt = prompt.replace("<INSERTANTWORT" + str(i) + ">",row["Answer "+ str(i+1)].replace(str(i) + ")",""))
        print(prompt_model(prompt))
        answer = prompt_model(prompt).split("<ASSISTENT>: BEGINANTWORT")[1].split("ENDANTWORT")
        found = False
        for i in range(5):
            if row["Answer "+ str(i+1)].replace(str(i) + ")","").strip().lower() == answer[0].strip().lower():
                found = i
                break
        #print(answer)
        if found != False:
            questions_df.loc[index, "Answer LLM"] = found
        questions_df.loc[index, "Prediction LLM"] = " ENDANTWORT ".join(answer)
print("Values set:", len(questions_df[questions_df["Answer LLM"] != -1]),"/",len(questions_df))


0it [00:00, ?it/s]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> Du bist ein hilfreicher Assistent, der alle Fragen so korrekt wie möglich beantwortet.<BENUTZER>: Es folgt eine Fallbeschreibung, dessen Anfang und Ende durch die TagsBEGINFALL undENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGSBEGINFRAGE undENDFRAGE steht. Die Antwortmöglichkeiten stehen jeweils zwischen den TagsBEGINANTWORT undENDANTWORT.BEGINFALL Eine 35-jährige Patientin wird in Begleitung des Notarztes in die Klinik gebracht. Die Patientin war auf ihrem Pferd geritten. Dies ging durch, und die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Bei Eintreffen des Notarztes GCS 8. Intubation vor Ort und Verbringen in das nächstgelegene Krankenhaus. Übergabe durch den Notarzt: intubierte Patientin, hochgradig volumenbedürftig mit hochlaufenden kreislaufwirksamen Medikamenten (Adrenalin). In der Untersuchung: Patientin im Stiffneck auf der Vakuummatratze, Pupillen anisokor, links größer als rechts. Li

0it [00:17, ?it/s]


IndexError: list index out of range

In [ ]:
questions_df.to_csv("leo_mistral_hessiani_output.csv", sep=";")

In [ ]:
y_pred = questions_df[questions_df["Answer LLM"] != -1]["Answer LLM"]
y_true = questions_df[questions_df["Answer LLM"] != -1]["Correct Answer"]

In [ ]:
print("Accuracy:", accuracy_score(y_true, y_pred))